# CSV to GeoDataframe with GeoPandas
If a CSV file includes coordinates - either a coordinate pair representing a point location, or a series of coordinate pairs depicting a line or a polygon's permiter - then we can use those coordinates to construct a geometric object and thus create a spatially enabled dataframe, which in Geopandas is referred to as a <u>geodataframe</u>. 

Here we focus on the steps involved in going from raw coordinate data stored in a field of CSV file to a spatial dataframe. In doing so, we discuss the hierarchy of components that go into adding spatial elements to a dataframe: from geometries, to geoseries, and finally to geodataframes.  

We'll start with the simplest example of creating a point spatial dataframe from a CSV file containing latitude and longitude coordinates. The data we'll use in this exercise is electric vehicle charging locations in North Carolina ([source](https://afdc.energy.gov/data_download)).

### 1. Constructing a Pandas dataframe from the CSV file
We'll use an API to fetch CSV data listing the electric vehicle charging locations in North Carolina and load that file directly into a familiar Pandas dataframe named `df_EVStations`.

In [10]:
#Import the requests object
import requests
import pandas as pd

In [14]:
#Construct the request
serviceURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.csv'
parameters = {
    'access':'all',
    'api_key':'oA9dHswdtlpAx5qLEdV1StM1mUB8KsgWluSfoEuL',
    'fuel_type':'ELEC',
    'status':'all',
    'state':'NC',
    'download':'true'
}

In [21]:
#Process the request
response = requests.get(serviceURL,parameters)
df_EVStations = pd.read_csv(response.url)

In [19]:
#Examine the columns, noting the data include "latitude"  "longitude" columns
df_EVStations.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

### 2. Creating geometries from latitude and longitude coordinates
Converting our simple lat/long values to a spatially enabled geometric object is a multistep process. First, we need to construct a new column in our dataframe that contains X and Y values as coordinate pairs (i.e. not as individual columns). From there, we can "upgrade" our dataframe to a *geo*dataframe using Geopandas functionality, specifying which data to use as the geometry column and also what projection our coordinate data uses.  

#### 2a. Creating a coordinate column from our individual X and Y columns
Here we create a new Pandas series object (i.e. a single column of data) consisting of a tuple of X and Y values found in each row of our dataframe. To combine values into a single object, we use the Python `zip` function used in the context of "list comprehension". 

In [41]:
#Add a new column to our dataset
for i,row in df_EVStations.head(4).iterrows():
    print (i,row['Latitude'])

0 35.2269135
1 35.778416
2 35.77435
3 35.392063


In [46]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
geometry[:5]

In [26]:
#Fetch the coordinates from our first row
theLat,theLng = df_EVStations.loc[0,['Latitude','Longitude']]
print(theLat,theLng)

35.2269135 -80.8501816


The above extracts two object, but what we want is actually a single object, a list containing the two coordinate values...

In [30]:
#Fetch the coordinates from our first row - AS A COORDINATE PAIR LIST OBJECT
theCoordinate = df_EVStations.loc[0,['Latitude','Longitude']]
theCoordinate

[35.2269135, -80.8501816]

Close, byt the above actually returns a Pandas _series_ object. If we apply the the `tolist()` function, we can convert the values to a list object...

In [31]:
#Convert theCoordinates from series to list
theCoordinate.tolist()

[35.2269135, -80.8501816]

So, all in one line it would be...

In [34]:
theCoordinate = df_EVStations.loc[0,['Latitude','Longitude']].tolist()
theCoordinate

[35.2269135, -80.8501816]

What we have first have to do is convert the two columns containing latitude and longitude into _one_ column containing a coordinate pair. To do this we use the Python `zip` function.

In [39]:
geometry = [Point(xy) for xy in zip(df]latitude], df.y)]

AttributeError: 'DataFrame' object has no attribute 'x'

In [36]:
coords = zip(df_EVStations['Longitude'],df_EVStations['Latitude'])
coords